### DESCRIPTION:
    This example shows how to retrieve data from Azure SQL DB by using GPT.  
    Asking questions in plain english that gets "translated" by GPT into SQL.
    Using Langchain SQLDatabaseToolkit
### REQUIREMENTS:
    1. Create an Azure SQL DB and populate it with data.
    2. Create an .env file in the root folder with the following variables:
      SQL_SERVER="<server>"
      SQL_USER="<user>"
      SQL_PWD="<pwd>"
      SQL_DBNAME="<dbname>"

### Sample questions you can ask:
      List the tables in the database
      How many products are in the Adventure Works database?
      How many Products are color black?
      How many SalesOrderDetail are for the Product AWC Logo Cap ?
      List the top 10 most expensive products
      What are the top 10 highest grossing products in the Adventure Works database?

### For more information about Langchain agent toolkits, see:
  https://github.com/hwchase17/langchain/tree/master/langchain/agents/agent_toolkits


In [ ]:
from langchain import SQLDatabase
from langchain.llms import AzureOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from dotenv import load_dotenv
import utils

In [ ]:
def ask_question(agent_executor, question):
    answer = agent_executor.run(question)
    pos = answer.find("Question:")
    if pos > 0:
        answer = answer[0:pos]
    pos = answer.find("<|im_end|>")
    if pos > 0:
        answer = answer[0:pos]
    print("Answer:", answer)

In [ ]:
utils.init_OpenAI()
llm = AzureOpenAI(deployment_name=utils.OPENAI_DEPLOYMENT_NAME, model_name=utils.OPENAI_MODEL_NAME)
sqlconn = f"mssql+pymssql://{utils.SQL_USER}:{utils.SQL_PWD}@{utils.SQL_SERVER}:1433/{utils.SQL_DBNAME}"
db = SQLDatabase.from_uri(sqlconn)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(llm=llm,toolkit=toolkit,verbose=False)

In [ ]:
ask_question(agent_executor, "How many products are in the Adventure Works database?")

In [ ]:
ask_question(agent_executor, "What is the average list price of the products in the Adventure Works database?")

In [ ]:
ask_question(agent_executor, "How many Products are color black?")

In [ ]:
ask_question(agent_executor, "How many SalesOrderDetail are for the Product AWC Logo Cap ?")

In [ ]:
ask_question(agent_executor, "List the top 10 most expensive products")

In [ ]:
ask_question(agent_executor, "What are the top 10 highest grossing products in the Adventure Works database?")